In [2]:
# import pandas as pd
# import ray
# from ray.data.datasource import FastFileMetadataProvider

In [3]:
# lj_df = pd.read_csv(
#     "https://uberduck-datasets-dirty.s3.us-west-2.amazonaws.com/lj_for_upload/metadata_formatted_100.txt",
#     sep="|",
#     header=None,
#     quoting=3,
#     names=["path", "transcript", "speaker_id"], # pitch path is implicit - this should be changed
# )

# paths = lj_df.path.tolist()
# transcripts = lj_df.transcript.tolist()
# speaker_ids = lj_df.speaker_id.tolist()

# parallelism_length = 400
# audio_ds = ray.data.read_binary_files(
#     paths,
#     parallelism=parallelism_length,
#     meta_provider=FastFileMetadataProvider(),
#     ray_remote_args={"num_cpus": 0.2},
# )
# audio_ds = audio_ds.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )

# paths = ray.data.from_items(paths, parallelism=parallelism_length)
# paths_ds = paths.map_batches(lambda x: x, batch_format="pyarrow", batch_size=None)

# transcripts = ray.data.from_items(transcripts, parallelism=parallelism_length)
# transcripts_ds = transcripts.map_batches(lambda x: x, batch_format="pyarrow", batch_size=None)

# speaker_ids_ds = ray.data.from_items(speaker_ids, parallelism=parallelism_length)
# speaker_ids_ds = speaker_ids_ds.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
# output_dataset = (
#     transcripts_ds.zip(audio_ds)
#     .zip(paths_ds)
#     .zip(speaker_ids_ds)
# )
# output_dataset = output_dataset.map_batches(
#     lambda table: table.rename(
#         columns={
#             "value_1": "transcript",
#             "value_2": "audio_bytes",
#             "value_3": "path",
#             "value_4": "speaker_id"
#         }
#     )
# )

In [2]:
2+2

4

In [1]:
from uberduck_ml_dev.exec.train_radtts_with_ray import get_ray_dataset, train_func
from ray.train.torch import TorchTrainer, TorchCheckpoint, TorchTrainer
class TorchCheckpointFixed(TorchCheckpoint):
    def __setstate__(self, state: dict):
        if "_data_dict" in state and state["_data_dict"]:
            state = state.copy()
            state["_data_dict"] = self._decode_data_dict(state["_data_dict"])
        super(TorchCheckpoint, self).__setstate__(state)



In [2]:
ray_dataset = get_ray_dataset()

2023-03-07 15:09:32,737	INFO worker.py:1360 -- Connecting to existing Ray cluster at address: 10.0.37.211:6379...
2023-03-07 15:09:32,765	INFO worker.py:1548 -- Connected to Ray cluster. View the dashboard at https://console.anyscale.com/api/v2/sessions/ses_ha6my3r3kl4mn9jdylcqnfutq1/services?redirect_to=dashboard 
2023-03-07 15:09:32,783	INFO packaging.py:330 -- Pushing file package 'gcs://_ray_pkg_eccc6979c79037b7d167507f1405cae4.zip' (6.81MiB) to Ray cluster...
2023-03-07 15:09:32,871	INFO packaging.py:343 -- Successfully pushed file package 'gcs://_ray_pkg_eccc6979c79037b7d167507f1405cae4.zip'.
(_get_read_tasks pid=3656, ip=10.0.48.48) 2023-03-07 15:09:34,374	WARNING file_meta_provider.py:162 -- Expanding 100 path(s). This may be a HIGH LATENCY operation on some cloud storage services. If the specified paths all point to files and never directories, try rerunning this read with `meta_provider=FastFileMetadataProvider()`.


In [3]:
# datum = ray_dataset.take(1)

In [4]:
from ray.air.config import ScalingConfig, RunConfig
from ray.tune import SyncConfig

In [5]:
from uberduck_ml_dev.exec.train_radtts_with_ray import train_config, model_config
train_config['n_group_size'] = model_config['n_group_size']
train_config['dur_model_config'] = model_config['dur_model_config']
train_config['f0_model_config'] = model_config['f0_model_config']
train_config['energy_model_config'] = model_config['energy_model_config']
train_config['v_model_config']=model_config['v_model_config']

In [6]:
trainer = TorchTrainer(
    train_loop_per_worker=train_func,
    train_loop_config=train_config,
    scaling_config=ScalingConfig(
        num_workers=2, use_gpu=True, resources_per_worker=dict(CPU=4, GPU=1)
    ),
    run_config=RunConfig(
        sync_config=SyncConfig(upload_dir="s3://uberduck-anyscale-data/checkpoints")
    ),
    datasets={"train": ray_dataset},
)

In [7]:
import os
os.environ[
        "TORCH_DISTRIBUTED_DEBUG"
    ] = "DETAIL"

In [ ]:
result = trainer.fit()

(RayTrainWorker pid=303, ip=10.0.5.7) 2023-03-07 15:09:46,491	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=2]
(TorchTrainer pid=3738, ip=10.0.48.48) 2023-03-07 15:09:46,610	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(<lambda>)] -> AllToAllOperator[zip] -> AllToAllOperator[zip] -> AllToAllOperator[zip] -> TaskPoolMapOperator[MapBatches(<lambda>)] -> AllToAllOperator[randomize_block_order]
(TorchTrainer pid=3738, ip=10.0.48.48) 2023-03-07 15:09:49,880	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[read] -> TaskPoolMapOperator[MapBatches(<lambda>)]
(TorchTrainer pid=3738, ip=10.0.48.48) 2023-03-07 15:09:55,809	WARNING plan.py:528 -- Warning: The Ray cluster currently does not have any available CPUs. The Dataset job will hang unless more CPUs are freed up. A common reason is that cluster resources are used by Actors or Tune trials; see the following link for mor

(autoscaler +26s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +26s) Adding 5 node(s) of type worker-node-type-0.


(TorchTrainer pid=3738, ip=10.0.48.48) /home/ray/anaconda3/lib/python3.10/site-packages/ray/train/_internal/dataset_iterator.py:64: UserWarning: session.get_dataset_shard returns a ray.data.DatasetIterator instead of a Dataset/DatasetPipeline as of Ray v2.3. Use iter_torch_batches(), to_tf(), or iter_batches() to iterate over one epoch. See https://docs.ray.io/en/latest/data/api/dataset_iterator.html for full DatasetIterator docs.
(TorchTrainer pid=3738, ip=10.0.48.48)   warnings.warn(
(RayTrainWorker pid=1124519) wandb: Currently logged in as: uberduck. Use `wandb login --relogin` to force relogin
(RayTrainWorker pid=303, ip=10.0.5.7) wandb: Currently logged in as: uberduck. Use `wandb login --relogin` to force relogin
(RayTrainWorker pid=1124519) wandb: wandb version 0.13.11 is available!  To upgrade, please run:
(RayTrainWorker pid=1124519) wandb:  $ pip install wandb --upgrade
(RayTrainWorker pid=1124519) wandb: Tracking run with wandb version 0.13.10
(RayTrainWorker pid=1124519) w

(RayTrainWorker pid=1124519) CUDA AVAILABLE:  True
(RayTrainWorker pid=1124519) Applying spectral norm to text encoder LSTM


(RayTrainWorker pid=1124519) /tmp/ray/session_2023-03-06_08-59-42_925397_141/runtime_resources/working_dir_files/_ray_pkg_eccc6979c79037b7d167507f1405cae4/uberduck_ml_dev/models/common.py:1516: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
(RayTrainWorker pid=1124519) The boolean parameter 'some' has been replaced with a string parameter 'mode'.
(RayTrainWorker pid=1124519) Q, R = torch.qr(A, some)
(RayTrainWorker pid=1124519) should be replaced with
(RayTrainWorker pid=1124519) Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2349.)
(RayTrainWorker pid=1124519)   W = torch.qr(torch.FloatTensor(c, c).normal_())[0]
(RayTrainWorker pid=1124519) /home/ray/anaconda3/lib/python3.10/site-packages/torch/functional.py:1682: UserWarning: torch.lu is deprecated in favor of torch.linalg.lu_factor / torch.linalg.lu_factor_ex and will be removed in a

...

(RayTrainWorker pid=303, ip=10.0.5.7) Loss: 66.75285339355469
(RayTrainWorker pid=1124519) Loss: 66.76640319824219


(RayTrainWorker pid=1124519) /home/ray/anaconda3/lib/python3.10/site-packages/torch/nn/modules/rnn.py:777: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:968.)
(RayTrainWorker pid=1124519)   result = _VF.lstm(input, batch_sizes, hx, self._flat_weights, self.bias,
(RayTrainWorker pid=303, ip=10.0.5.7) /home/ray/anaconda3/lib/python3.10/site-packages/torch/nn/modules/rnn.py:777: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:968.)
(RayTrainWorker pid=303, ip=10.0.5.7)   result = _VF.lstm(input, batch_sizes,

(RayTrainWorker pid=303, ip=10.0.5.7) Loss: 66.75389099121094
(RayTrainWorker pid=1124519) Loss: 66.77154541015625


(RayTrainWorker pid=303, ip=10.0.5.7) /home/ray/anaconda3/lib/python3.10/site-packages/torch/nn/modules/rnn.py:777: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:968.)
(RayTrainWorker pid=303, ip=10.0.5.7)   result = _VF.lstm(input, batch_sizes, hx, self._flat_weights, self.bias,
(RayTrainWorker pid=1124519) /home/ray/anaconda3/lib/python3.10/site-packages/torch/nn/modules/rnn.py:777: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:968.)
(RayTrainWorker pid=1124519)   result = _VF.lstm(input, batch_sizes,

(RayTrainWorker pid=303, ip=10.0.5.7) Loss: 66.65056610107422
(RayTrainWorker pid=1124519) Loss: 66.45819854736328
(autoscaler +6m24s) Removing 2 nodes of type worker-node-type-0 (idle).
(autoscaler +6m28s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +6m34s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +6m38s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +6m44s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +6m48s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +6m54s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +6m59s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +7m4s) Removing 5 nodes of type worker-node-type-0 (idle).
(autoscaler +7m8s) Removing 5 nodes of type worker-node-type-0 (idle).


In [9]:
??librosa_mel_fn

Signature:
librosa_mel_fn(
    sr,
    n_fft,
    n_mels=128,
    fmin=0.0,
    fmax=None,
    htk=False,
    norm='slaney',
    dtype=<class 'numpy.float32'>,
)
Source:   
@cache(level=10)
def mel(
    sr,
    n_fft,
    n_mels=128,
    fmin=0.0,
    fmax=None,
    htk=False,
    norm="slaney",
    dtype=np.float32,
):
    """Create a Mel filter-bank.

    This produces a linear transformation matrix to project
    FFT bins onto Mel-frequency bins.

    Parameters
    ----------
    sr        : number > 0 [scalar]
        sampling rate of the incoming signal

    n_fft     : int > 0 [scalar]
        number of FFT components

    n_mels    : int > 0 [scalar]
        number of Mel bands to generate

    fmin      : float >= 0 [scalar]
        lowest frequency (in Hz)

    fmax      : float >= 0 [scalar]
        highest frequency (in Hz).
        If `None`, use ``fmax = sr / 2.0``

    htk       : bool [scalar]
        use HTK formula instead of Slaney

    norm : {None, 'slaney', or num

In [1]:
from librosa.filters import mel as librosa_mel_fn
help(librosa_mel_fn)

Help on function mel in module librosa.filters:

mel(sr, n_fft, n_mels=128, fmin=0.0, fmax=None, htk=False, norm='slaney', dtype=<class 'numpy.float32'>)
    Create a Mel filter-bank.
    
    This produces a linear transformation matrix to project
    FFT bins onto Mel-frequency bins.
    
    Parameters
    ----------
    sr        : number > 0 [scalar]
        sampling rate of the incoming signal
    
    n_fft     : int > 0 [scalar]
        number of FFT components
    
    n_mels    : int > 0 [scalar]
        number of Mel bands to generate
    
    fmin      : float >= 0 [scalar]
        lowest frequency (in Hz)
    
    fmax      : float >= 0 [scalar]
        highest frequency (in Hz).
        If `None`, use ``fmax = sr / 2.0``
    
    htk       : bool [scalar]
        use HTK formula instead of Slaney
    
    norm : {None, 'slaney', or number} [scalar]
        If 'slaney', divide the triangular mel weights by the width of the mel band
        (area normalization).
    
      

In [1]:
import librosa
from librosa.util import pad_center
librosa.__version__

'0.10.0'

In [2]:
# import pad_center
help(pad_center)

Help on function pad_center in module librosa.util.utils:

pad_center(data: 'np.ndarray', *, size: 'int', axis: 'int' = -1, **kwargs: 'Any') -> 'np.ndarray'
    Pad an array to a target length along a target axis.
    
    This differs from `np.pad` by centering the data prior to padding,
    analogous to `str.center`
    
    Examples
    --------
    >>> # Generate a vector
    >>> data = np.ones(5)
    >>> librosa.util.pad_center(data, size=10, mode='constant')
    array([ 0.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.])
    
    >>> # Pad a matrix along its first dimension
    >>> data = np.ones((3, 5))
    >>> librosa.util.pad_center(data, size=7, axis=0)
    array([[ 0.,  0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.,  0.],
           [ 1.,  1.,  1.,  1.,  1.],
           [ 1.,  1.,  1.,  1.,  1.],
           [ 1.,  1.,  1.,  1.,  1.],
           [ 0.,  0.,  0.,  0.,  0.],
           [ 0.,  0.,  0.,  0.,  0.]])
    >>> # Or its second dimension
    >>> librosa.util.pad_cent

In [3]:
import numpy as np
pad_center(np.ones(1000), 5000)

TypeError: pad_center() takes 1 positional argument but 2 were given

In [5]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install librosa==0.8.1

  Using cached librosa-0.8.1-py3-none-any.whl (203 kB)
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.0
    Uninstalling librosa-0.10.0:
      Successfully uninstalled librosa-0.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tts 0.11.1 requires cython==0.29.28, but you have cython 0.29.33 which is incompatible.
tts 0.11.1 requires inflect==5.6.0, but you have inflect 6.0.2 which is incompatible.
tts 0.11.1 requires librosa==0.8.0, but you have librosa 0.8.1 which is incompatible.
tts 0.11.1 requires numpy==1.22.4; python_version == "3.10", but you have numpy 1.22.0 which is incompatible.


In [2]:
import pandas as pd
import ray
parallelism_length = 400

In [3]:
lj_df = pd.read_csv(
    "https://uberduck-datasets-dirty.s3.us-west-2.amazonaws.com/lj_for_upload/metadata_formatted_100_edited.txt",
    sep="|",
    header=None,
    quoting=3,
    names=["path", "transcript", "speaker_id"], # pitch path is implicit - this should be changed
)
speaker_ids = lj_df.speaker_id.tolist()
speaker_ids_ds = ray.data.from_items(speaker_ids, parallelism=parallelism_length)
speaker_ids_ds = speaker_ids_ds.map_batches(
    lambda x: x, batch_format="pyarrow", batch_size=None
)

2023-03-06 16:54:26,264	INFO worker.py:1360 -- Connecting to existing Ray cluster at address: 10.0.37.211:6379...
2023-03-06 16:54:26,291	INFO worker.py:1548 -- Connected to Ray cluster. View the dashboard at https://console.anyscale.com/api/v2/sessions/ses_ha6my3r3kl4mn9jdylcqnfutq1/services?redirect_to=dashboard 
2023-03-06 16:54:26,316	INFO packaging.py:330 -- Pushing file package 'gcs://_ray_pkg_c450220a9fe1382fee31ab05c616a7cf.zip' (6.75MiB) to Ray cluster...
2023-03-06 16:54:26,438	INFO packaging.py:343 -- Successfully pushed file package 'gcs://_ray_pkg_c450220a9fe1382fee31ab05c616a7cf.zip'.


In [4]:
lj_df

,path,transcript,speaker_id
0,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,"Printing, in the only sense with which we are ...",0
1,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,in being comparatively modern.,0
2,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,For although the Chinese took impressions from...,0
3,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,"produced the block books, which were the immed...",0
4,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,the invention of movable metal letters in the ...,0
...,...,...,...
95,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,have now come into general use and are obvious...,0
96,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,a little reduced in ugliness. The design of th...,0
97,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,"and the whole effect is a little too gray, owi...",0
98,s3://uberduck-datasets-dirty/lj_for_upload/lj_...,"It must be remembered, however, that most mode...",0


In [3]:
output_dataset = speaker_ids_ds.map_batches(
    lambda table: table.rename(
        columns={

            "value_1": "speaker_id"
        }
    )
)

In [4]:
output_dataset.speaker_id.tolist()

AttributeError: 'Dataset' object has no attribute 'speaker_id'

(autoscaler +2s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +2s) Removing 2 nodes of type worker-node-type-0 (idle).
(autoscaler +7s) Removing 2 nodes of type worker-node-type-0 (idle).
(autoscaler +12s) Removing 2 nodes of type worker-node-type-0 (idle).


In [ ]:
pd.DataFrame(output_dataset)

In [ ]:
output_dataset

2023-03-06 16:50:22,749	INFO bulk_executor.py:41 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[MapBatches(<lambda>)->MapBatches(<lambda>)]
MapBatches(<lambda>)->MapBatches(<lambda>): 100%|██████████| 100/100 [00:03<00:00, 28.88it/s]


In [39]:
#     lj_df = pd.read_csv(
#         "https://uberduck-datasets-dirty.s3.us-west-2.amazonaws.com/vctk_mic1/all_with_embs.txt",
#         sep="|",
#         header=None,
#         quoting=3,
#         names=["path", "speaker_id", "transcript", "dataset_audio_file_id", "emb_path"],
#     )

In [38]:
# lj_df['path'][0]

In [37]:
# transcripts_ds = ray.data.from_items(transcripts, parallelism=parallelism_length)
# speaker_ids_ds = ray.data.from_items(speaker_ids, parallelism=parallelism_length)
# dataset_audio_file_ids = ray.data.from_items(
#     dataset_audio_files, parallelism=parallelism_length
# )

# pitch_paths_ds = ray.data.read_binary_files(
#     pitch_paths,
#     parallelism=parallelism_length,
#     meta_provider=FastFileMetadataProvider(),
#     ray_remote_args={"num_cpus": 0.2},
# )

# audio_ds = audio_ds.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
# transcripts_ds = transcripts_ds.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
# dataset_audio_file_ids = dataset_audio_file_ids.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
# paths_ds = paths.map_batches(lambda x: x, batch_format="pyarrow", batch_size=None)
# pitches_paths_ds = pitch_paths_ds.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
# speaker_ids_ds = speaker_ids_ds.map_batches(
#     lambda x: x, batch_format="pyarrow", batch_size=None
# )
